## Some movies are more equal than others

In [1]:
import numpy as np

In [2]:
from hw12 import *

With offsets [[ 0.22024566]
 [-0.22193986]] 0.0076238881829543415
With no offsets [[0.50148126]
 [0.0562376 ]]
ALS result for k = 2 : rmse train = [[0.00200034]] ; rmse validate = [[0.00200034]]
SGD result for k = 2 : rmse train = [[3.64426549]] ; rmse validate = [[3.64426549]]


In [3]:
Z = np.array([[1], [1], [5], [1], [5], [5], [1]])
b_v = np.array([[3], [3], [3], [3], [3], [5], [1]])
B = np.array([[1, 10], [1, 10], [10, 1], [1, 10], [10, 1], [5, 5], [5, 5]])

In [4]:
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)
u_a, b_u_a
# Solution with offsets, using ridge_analytic provided in code file
# (array([[ 0.22024566],
#         [-0.22193986]]),
#  array([[ 0.00762389]]))

(array([[ 0.22024566],
        [-0.22193986]]), 0.0076238881829543415)

In [5]:
u_a_no_b = np.dot(np.linalg.inv(np.dot(B.T, B) + 1 * np.identity(2)), np.dot(B.T, Z))
u_a_no_b
# Solution using previous model, with no offsets
# array([[ 0.50148126],
#        [ 0.0562376 ]])

array([[0.50148126],
       [0.0562376 ]])

In [6]:
10 * 0.22024566 + 1 * -0.22193986 + 0.00762389

1.9881406300000002

In [7]:
10 * 0.50148126 + 1 * 0.0562376

5.071050200000001

In [8]:
1 * 0.22024566 + 10 * -0.22193986 + 0.00762389 + 5

3.0084709500000004

In [9]:
1 * 0.50148126 + 10 * 0.0562376

1.06385726

In [68]:
data = load_ratings_data()
movies_dict, genres_dict = load_movies()
model = mf_als(data, None, k=10, lam=1, max_iter=20)
save_model(model)
# model = load_model()

Loading from ratings.csv users 13366 items 2000
Iteration 1 finished. Total Elapsed Time: 5.91
Iteration 2 finished. Total Elapsed Time: 11.34
Iteration 3 finished. Total Elapsed Time: 16.94
Iteration 4 finished. Total Elapsed Time: 22.27
Iteration 5 finished. Total Elapsed Time: 27.72
Iteration 6 finished. Total Elapsed Time: 33.49
Iteration 7 finished. Total Elapsed Time: 39.11
Iteration 8 finished. Total Elapsed Time: 44.15
Iteration 9 finished. Total Elapsed Time: 49.23
Iteration 10 finished. Total Elapsed Time: 54.31
Iteration 11 finished. Total Elapsed Time: 59.40
Iteration 12 finished. Total Elapsed Time: 64.67
Iteration 13 finished. Total Elapsed Time: 70.21
Iteration 14 finished. Total Elapsed Time: 75.31
Iteration 15 finished. Total Elapsed Time: 80.23
Iteration 16 finished. Total Elapsed Time: 85.25
Iteration 17 finished. Total Elapsed Time: 90.13
Iteration 18 finished. Total Elapsed Time: 95.27
Iteration 19 finished. Total Elapsed Time: 100.26
Iteration 20 finished. Total E

In [11]:
sorted(list(map(int, set(np.array(data)[:, 0])))).index(270894)

13365

In [12]:
len(genres_dict.keys())

2000

In [13]:
all_ratings = list(filter(lambda x: x[0] == 270894, data))

In [14]:
from collections import Counter
c = Counter()
for user, movie, rating in all_ratings:
    if rating == 5.0:
        genres = genres_dict[movie]
        for genre in genres:
            c[genre] += 1

In [15]:
c

Counter({'Action': 20,
         'Adventure': 46,
         'Animation': 74,
         'Children': 58,
         'Comedy': 42,
         'Crime': 6,
         'Drama': 27,
         'Fantasy': 31,
         'Horror': 3,
         'IMAX': 12,
         'Musical': 23,
         'Mystery': 4,
         'Romance': 16,
         'Sci-Fi': 15,
         'Thriller': 8,
         'War': 2})

In [16]:
u, b_u, v, b_v = model

In [17]:
u[13365]

array([[ 0.53310525],
       [ 0.64521269],
       [ 1.13737098],
       [ 0.86048265],
       [-0.77917892],
       [ 0.2059275 ],
       [ 1.36139392],
       [-0.3436632 ],
       [-0.42036595],
       [-0.63237646]])

In [18]:
movie_ratings = []
for i, movie in enumerate(v):
    movie_ratings.append(np.dot(u[270894].T, movie) + b_u[270894] + b_v[i])

In [19]:
ratings = []
for i, movie in enumerate(movie_ratings):
    ratings.append((i, np.asscalar(movie)))

In [20]:
sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)

In [21]:
top_movies = map(lambda x: x[0], sorted_ratings)

In [22]:
real_top_movies = []
for top in top_movies:
    if top in movies_dict:
        real_top_movies.append(top)

In [23]:
len(v), len(u)

(166529, 270895)

In [24]:
rated_movies = set(map(lambda x: x[1], all_ratings))
count = 0
for m in real_top_movies[:50]:
    genres = genres_dict[m]
    if 'Animation' in genres and m not in rated_movies:
        count += 1
print(count)

13


In [25]:
genres_dict[745]

['Animation', 'Children', 'Comedy']

In [74]:
def most_similar_movies(movie_id):
    similarities = []
    for other_movie_id, features in enumerate(v):
        if other_movie_id == movie_id:
            continue
        if other_movie_id not in movies_dict:
            continue
        similarity = np.asscalar(np.dot(v[movie_id].T, features) / np.linalg.norm(v[movie_id]) / np.linalg.norm(features))
        similarities.append((other_movie_id, similarity))
    return sorted(similarities, key=lambda x: x[1], reverse=True)

In [45]:
list(map(lambda x: x[0], most_similar_movies(260)[:10]))

[1196, 1210, 1291, 1198, 122886, 166528, 480, 2414, 2115, 2628]

In [75]:
list(map(lambda x: x[0], most_similar_movies(2628)[:10]))

[5378, 33493, 1027, 1210, 86, 122886, 110, 5952, 7153, 51662]

In [70]:
for m, s in most_similar_movies(2628)[:10]:
    print(movies_dict[m], m, s)

Star Wars: Episode II - Attack of the Clones (2002) 5378 0.9650331205405259
Star Wars: Episode III - Revenge of the Sith (2005) 33493 0.9507111671419216
Robin Hood: Prince of Thieves (1991) 1027 0.8105825590158939
Star Wars: Episode VI - Return of the Jedi (1983) 1210 0.792485960984933
White Squall (1996) 86 0.7769841747234759
Star Wars: Episode VII - The Force Awakens (2015) 122886 0.7577302404552065
Braveheart (1995) 110 0.7480259791240388
"Lord of the Rings: The Two Towers, The (2002)" 5952 0.7440763075249501
"Lord of the Rings: The Return of the King, The (2003)" 7153 0.7352507159630771
300 (2007) 51662 0.7238528106736651


In [69]:
for m, s in most_similar_movies(260)[:10]:
    print(movies_dict[m], m, s)

Star Wars: Episode V - The Empire Strikes Back (1980) 1196 0.9944192698935015
Star Wars: Episode VI - Return of the Jedi (1983) 1210 0.9629512279132393
Indiana Jones and the Last Crusade (1989) 1291 0.8919338582521604
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) 1198 0.861313102178545
Star Wars: Episode VII - The Force Awakens (2015) 122886 0.8528360607190725
Rogue One: A Star Wars Story (2016) 166528 0.7564123473977342
Jurassic Park (1993) 480 0.7406354405156786
Young Sherlock Holmes (1985) 2414 0.7256168733918685
Indiana Jones and the Temple of Doom (1984) 2115 0.7140374507269162
Star Wars: Episode I - The Phantom Menace (1999) 2628 0.7132885063037011


In [40]:
list(map(lambda x: x[0], most_similar_movies(2628)[-10:]))

[2303, 1883, 2118, 8949, 52, 2863, 562, 2599, 60950, 2759]

In [82]:
sim_sum = 0
sim_cnt = 0
for movie1 in movies_dict.keys():
    for movie2 in movies_dict.keys():
        if movie1 == movie2:
            continue
        similarity = np.asscalar(np.dot(v[movie1].T, v[movie2]) / np.linalg.norm(v[movie1]) / np.linalg.norm(v[movie2]))
        sim_sum += similarity
        sim_cnt += 1

In [83]:
np.mean(sim_sum / sim_cnt)

0.0024403534965481387

In [79]:
len(movies_dict.keys())

2000

In [85]:
from collections import defaultdict
movie_genres = defaultdict(list)
for movie in movies_dict:
    genres = genres_dict[movie]
    for genre in genres:
        movie_genres[genre].append(movie)

In [88]:
for genre, movies in movie_genres.items():
    sim_sum = 0
    sim_cnt = 0
    for movie1 in movies:
        for movie2 in movies:
            if movie1 == movie2:
                continue
            similarity = np.asscalar(np.dot(v[movie1].T, v[movie2]) / np.linalg.norm(v[movie1]) / np.linalg.norm(v[movie2]))
            sim_sum += similarity
            sim_cnt += 1
    print(genre, sim_sum / sim_cnt)

Adventure 0.07685861183679463
Animation 0.32625652925321
Children 0.2697289620582803
Comedy 0.06444205000037817
Fantasy 0.08992929956315779
Romance 0.06777477960424376
Drama 0.05902013563906724
Action 0.12284295495599863
Crime 0.05001755733335963
Thriller 0.06354218925995296
Horror 0.23624274707376675
Mystery 0.05073690338449419
Sci-Fi 0.13921973228743367
War 0.137469084041903
Musical 0.2932409088196969
IMAX 0.27294038771762075
Documentary 0.4196998123108044
Western 0.15054916258906906
Film-Noir 0.4471172819305044
